<a href="https://colab.research.google.com/github/Wn013/code-love/blob/main/Fanxing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copy Folder Google Drive to Google Drive - 1TouchPro

In [ ]:
# @title Run (Optimized Version - Skip Non-Retryable 403 + Download/Upload Fallback)
import os
import time
import re
import sys
import typing as t
import math
import random
import io # <<< THÊM: Để xử lý stream dữ liệu
# import json # Có thể không cần nếu error.reason hoạt động tốt
from googleapiclient.discovery import build, Resource
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload # <<< THÊM: Để download/upload
from google.colab import auth
from google.colab import drive
from tqdm.notebook import tqdm

# --- Constants ---
FOLDER_MIME_TYPE = 'application/vnd.google-apps.folder'
MAX_RETRIES = 5 # Số lần thử lại tối đa cho các lỗi CÓ THỂ retry
INITIAL_RETRY_DELAY = 1.0 # Thời gian chờ ban đầu (giây)

class DownloadFromDrive:
    # Sử dụng type hints để rõ ràng hơn về kiểu dữ liệu
    def __init__(self):
        self._total_size: float = 0.0 # Tổng kích thước đã copy (MB)
        self._limit_size: float = 0.0 # Giới hạn kích thước (GB) - sẽ được đặt ở phần Main
        self.excluded_strings: t.List[str] = []

    def get_user_credential(self) -> t.Optional[Resource]:
        """Xác thực người dùng và trả về đối tượng Drive service."""
        try:
            auth.authenticate_user()
            drive_service: Resource = build('drive', 'v3', cache_discovery=False)
            return drive_service
        except Exception as e:
            print(f"Lỗi khi xác thực hoặc tạo Drive service: {e}")
            return None

    def get_childs_from_folder(self, drive_service: Resource, folder_id: str, from_page: int, to_page: int) -> t.List[t.Dict[str, t.Any]]:
        """Lấy danh sách các mục con từ một thư mục, hỗ trợ phân trang và loại trừ."""
        files: t.List[t.Dict[str, t.Any]] = []
        page_token: t.Optional[str] = None
        query = f"'{folder_id}' in parents and trashed = false"
        if self.excluded_strings:
            not_contains_query = " and ".join([f"not name contains '{ext}'" for ext in self.excluded_strings])
            query += f" and {not_contains_query}"

        pages = 0
        print(f"Fetching file list from folder ID: {folder_id}...")
        while True:
            try:
                pages += 1
                # Tăng pageSize lên 1000 để giảm số lượng API call
                # Thêm 'size' và 'capabilities' vào fields để có thể kiểm tra trước
                response = drive_service.files().list(
                    q=query,
                    orderBy='name, createdTime',
                    fields='files(id, name, mimeType, size, capabilities(canDownload, canCopy)), nextPageToken', # <<< ĐÃ THÊM capabilities VÀ size
                    pageToken=page_token,
                    supportsAllDrives=True,
                    includeItemsFromAllDrives=True,
                    pageSize=1000 # Tối đa 1000
                ).execute()

                current_files = response.get('files', [])
                print(f"Page {pages}: Found {len(current_files)} items.")
                if to_page == 0 or (from_page <= pages <= to_page):
                    files.extend(current_files)

                page_token = response.get('nextPageToken', None)
                if page_token is None or (to_page > 0 and pages >= to_page):
                    print("Reached end of file list or target page.")
                    break
            except HttpError as error:
                print(f"An HTTP error occurred while listing files (Page {pages}): {error}")
                break # Thoát vòng lặp nếu có lỗi khi list file
            except Exception as e:
                print(f"An unexpected error occurred while listing files (Page {pages}): {str(e)}")
                break # Thoát vòng lặp
        print(f"Total files/folders fetched in specified range: {len(files)}")
        return files


    # --- HÀM SAO CHÉP VỚI RETRY LOGIC - ĐÃ SỬA ĐỔI ĐỂ BỎ QUA LỖI 403 KHÔNG THỂ RETRY ---
    def _execute_with_retry(self, api_call: t.Callable[[], t.Any]) -> t.Optional[t.Dict[str, t.Any]]:
        """Thực thi API call với retry. Bỏ qua ngay lỗi 403 không thể retry (quyền, policy...)."""
        retry_delay = INITIAL_RETRY_DELAY
        last_error = None # Lưu lỗi cuối cùng để hiển thị nếu hết retry
        for attempt in range(MAX_RETRIES):
            try:
                return api_call() # Thực thi hàm API
            except HttpError as error:
                last_error = error # Cập nhật lỗi cuối cùng
                status_code = error.resp.status
                reason = getattr(error, 'reason', None)
                # Cố gắng parse JSON nếu reason không rõ ràng (ít dùng vì reason thường đủ)
                # ... (có thể thêm logic parse JSON nếu cần) ...

                error_info_str = f"{status_code}{f' ({reason})' if reason else ''}"

                # --- LỖI 403: Kiểm tra lý do ---
                if status_code == 403:
                    non_retryable_403_reasons = [
                        'insufficientFilePermissions','insufficientPermissions',
                        'domainPolicy', 'fileOwnerNotMemberOfTeamDrive', 'cannotCopyFile',
                        'forbidden', 'dailyLimitExceeded', 'userAccessRevoked',
                        'teamDriveMembershipRequired', 'appNotAuthorizedToFile', 'sharingNotAllowed',
                        'teamDriveFileLimitExceeded', # Thêm giới hạn Team Drive
                        'organizationRestrictions', # Thêm giới hạn tổ chức
                    ]
                    retryable_403_reasons = [
                         'userRateLimitExceeded', 'rateLimitExceeded', 'sharingRateLimitExceeded',
                         # backendError đôi khi trả về 403 nhưng có thể retry
                         'backendError'
                    ]

                    if reason in non_retryable_403_reasons:
                        print(f"--- Skipping: Non-retryable 403 error: {error_info_str}.")
                        return None # Bỏ qua ngay
                    elif reason in retryable_403_reasons:
                        print(f"--- API Error ({error_info_str}) on attempt {attempt + 1}/{MAX_RETRIES}. Retrying in {retry_delay:.2f}s...")
                        # Tiếp tục vòng lặp để retry
                    else: # Lý do 403 không xác định -> Bỏ qua cho an toàn
                        print(f"--- Skipping: Uncertain 403 error reason: {error_info_str}.")
                        return None # Bỏ qua ngay

                # --- CÁC LỖI KHÁC CÓ THỂ THỬ LẠI (429 - Too Many Requests, 5xx - Server Errors) ---
                elif status_code in [429, 500, 502, 503, 504]:
                     print(f"--- API Error ({error_info_str}) on attempt {attempt + 1}/{MAX_RETRIES}. Retrying in {retry_delay:.2f}s...")
                     # Tiếp tục vòng lặp để retry
                # --- CÁC LỖI KHÁC: Không thử lại ---
                else:
                    print(f"--- Skipping: Unrecoverable HTTP error: {error_info_str}.")
                    return None # Bỏ qua ngay

                # --- LOGIC THỬ LẠI (Chỉ chạy nếu lỗi thuộc nhóm retryable) ---
                time.sleep(retry_delay)
                retry_delay = min(retry_delay * 2 + (random.random() * 0.5), 60) # Exponential backoff

            except Exception as e: # Các lỗi không phải HTTP
                last_error = e # Cập nhật lỗi cuối cùng
                print(f"--- Skipping: An unexpected non-HTTP error occurred during API call: {e}")
                # import traceback # Bỏ comment nếu cần debug sâu
                # traceback.print_exc()
                return None # Bỏ qua

        # Nếu vòng lặp kết thúc mà chưa thành công (đã retry hết số lần cho lỗi retryable)
        print(f"--- Skipping: API call failed after {MAX_RETRIES} attempts. Last error: {last_error}")
        return None # Trả về None để báo hiệu thất bại sau khi đã retry

    # ==============================================================
    # HÀM COPY_FILE - ĐÃ SỬA ĐỂ CÓ DOWNLOAD/UPLOAD FALLBACK
    # ==============================================================
    def copy_file(self, drive_service: Resource, dest_folder_id: str, source_file: t.Dict[str, t.Any]):
        """Sao chép một tệp hoặc xử lý thư mục con (đệ quy).
        Thử download và upload lại nếu copy trực tiếp bị chặn nhưng download được phép."""
        file_name = source_file.get('name', 'Unknown File')
        file_id = source_file.get('id')
        file_mime_type = source_file.get('mimeType')
        capabilities = source_file.get('capabilities', {}) # Lấy capabilities
        # Cố gắng lấy size, mặc định là 0 nếu không có
        filesize = int(source_file.get('size', 0))

        if not file_id:
            print(f"--- Skipping item [{file_name}] - Missing file ID.")
            return

        # --- Xử lý trường hợp là FILE ---
        if file_mime_type != FOLDER_MIME_TYPE:
            # *** KIỂM TRA TRƯỚC KHI THỰC HIỆN ***
            can_copy = capabilities.get('canCopy', True) # Mặc định là True nếu không có thông tin
            can_download = capabilities.get('canDownload', True) # Mặc định là True

            # Kiểm tra file tồn tại chính xác bằng tên và mimeType trong thư mục đích
            if self.check_if_exists(drive_service, dest_folder_id, file_name, file_mime_type):
                # Cập nhật size nếu file tồn tại (để tính toán tổng size chính xác hơn nếu dùng limit)
                # Tuy nhiên, để đơn giản, bỏ qua việc cập nhật size ở đây nếu chỉ cần check tồn tại
                # filesize = int(source_file.get('size', 0)) # Lấy lại size gốc để tính toán nếu cần
                # if filesize > 0:
                #     self._total_size += filesize / (1024 * 1024) # Cần thận trọng nếu file lớn và limit nhỏ
                print(f"[{file_name}] exists in destination. Skipping.")
                return # Bỏ qua nếu đã tồn tại

            # === ƯU TIÊN COPY TRỰC TIẾP (NHANH HƠN) NẾU ĐƯỢC PHÉP ===
            if can_copy:
                print(f"Attempting direct copy (fast): [{file_name}] (ID: {file_id})")
                body_file_inf = {'parents': [dest_folder_id], 'name': file_name}

                def copy_api_call():
                    return drive_service.files().copy(
                        body=body_file_inf,
                        fileId=file_id,
                        supportsAllDrives=True,
                        fields='id, name, size' # Lấy size từ file đã copy nếu gốc không có
                    ).execute()

                start_time = time.time()
                copied_file_info = self._execute_with_retry(copy_api_call) # Dùng retry đã sửa
                end_time = time.time()

                if copied_file_info:
                    # Lấy size chính xác hơn từ file đã copy nếu size gốc = 0
                    copied_size = int(copied_file_info.get('size', 0))
                    effective_size = copied_size if copied_size > 0 else filesize # Ưu tiên size đã copy
                    effective_size_mb = effective_size / (1024 * 1024)

                    if effective_size > 0:
                        self._total_size += effective_size_mb # Chỉ cộng size vào tổng khi thành công
                        duration = end_time - start_time
                        speed_mb = effective_size_mb / duration if duration > 0.001 else 0
                        print(f"[{file_name}] copied directly. Size [{effective_size_mb:0.2f}] MB. Speed [{speed_mb:0.2f}] MB/s")
                    else:
                        print(f"[{file_name}] copied directly (size 0 or unknown).")

                    # Kiểm tra giới hạn dung lượng
                    if self._limit_size > 0 and self._total_size >= (self._limit_size * 1024):
                        self.on_total_size_exceeded(f"Total size {self._total_size:.2f} MB exceeds limit {self._limit_size} GB. Ending the program.")
                    return # <<< QUAN TRỌNG: Kết thúc xử lý file này nếu copy trực tiếp thành công

                else:
                    # Lỗi đã được log trong _execute_with_retry
                    print(f"--- Direct copy failed for [{file_name}] (ID: {file_id}). See previous error logs.")
                    # Quan trọng: Nếu copy trực tiếp thất bại (dù là lỗi gì sau khi retry),
                    # chúng ta không nên thử download/upload nữa vì có thể cùng lý do (quyền, policy).
                    # Trừ khi muốn thử bất chấp, nhưng dễ gây lỗi lặp hoặc không cần thiết.
                    return # <<< Dừng xử lý file này nếu copy trực tiếp thất bại

            # === THỬ DOWNLOAD/UPLOAD NẾU KHÔNG COPY TRỰC TIẾP ĐƯỢC NHƯNG DOWNLOAD ĐƯỢC ===
            elif can_download:
                print(f"--- Direct copy disabled for [{file_name}]. Attempting download/upload (slower)...")

                # --- BƯỚC 1: DOWNLOAD TỪ NGUỒN VỀ COLAB LOCAL STORAGE ---
                # Tạo tên file tạm an toàn hơn (tránh ký tự đặc biệt)
                safe_file_name_part = re.sub(r'[\\/*?:"<>|]', "_", file_name)
                local_temp_path = f"/content/{file_id}_{safe_file_name_part}"
                print(f"   Downloading [{file_name}] (Size: {filesize / (1024*1024):.2f} MB) to Colab temp: {os.path.basename(local_temp_path)}")
                start_time_dl = time.time()
                download_success = False
                try:
                    request = drive_service.files().get_media(fileId=file_id, supportsAllDrives=True)
                    # Sử dụng io.FileIO để ghi file hiệu quả
                    with io.FileIO(local_temp_path, 'wb') as fh:
                        # chunksize lớn hơn có thể nhanh hơn nhưng tốn RAM hơn
                        downloader = MediaIoBaseDownload(fh, request, chunksize=32*1024*1024) # chunk 32MB
                        done = False
                        # Chỉ hiện progress bar nếu biết size và size > 0
                        disable_tqdm = (filesize <= 0)
                        progress = tqdm(total=filesize, unit='B', unit_scale=True, desc=f"   DL {file_name[:20]}...", leave=False, disable=disable_tqdm)
                        last_progress = 0
                        while not done:
                            status, done = downloader.next_chunk()
                            if status and not disable_tqdm:
                                current_progress = int(status.resumable_progress)
                                progress.update(current_progress - last_progress) # Cập nhật số byte đã tải thêm
                                last_progress = current_progress
                        progress.close()
                    end_time_dl = time.time()
                    duration_dl = end_time_dl - start_time_dl
                    if filesize > 0 and duration_dl > 0.001:
                         speed_dl_mb = (filesize / (1024*1024)) / duration_dl
                         print(f"   Download complete. Time: {duration_dl:.2f}s. Avg Speed: {speed_dl_mb:.2f} MB/s.")
                    else:
                         print(f"   Download complete. Time: {duration_dl:.2f}s.")
                    download_success = True
                except HttpError as error:
                    print(f"   --- Download failed for [{file_name}]: {error}")
                    # Xóa file tạm nếu có lỗi download
                    if os.path.exists(local_temp_path):
                        try: os.remove(local_temp_path)
                        except OSError: pass # Bỏ qua nếu không xóa được
                except Exception as e:
                    print(f"   --- An unexpected error occurred during download of [{file_name}]: {e}")
                    if os.path.exists(local_temp_path):
                        try: os.remove(local_temp_path)
                        except OSError: pass

                # --- BƯỚC 2: UPLOAD TỪ COLAB LÊN THƯ MỤC ĐÍCH ---
                if download_success:
                    # Kiểm tra lại dung lượng file đã tải về (phòng trường hợp size gốc không đúng)
                    try:
                        actual_downloaded_size = os.path.getsize(local_temp_path)
                        if actual_downloaded_size != filesize and filesize > 0:
                             print(f"   Note: Downloaded file size ({actual_downloaded_size / (1024*1024):.2f} MB) differs from source metadata ({filesize / (1024*1024):.2f} MB). Using actual size.")
                             filesize = actual_downloaded_size # Sử dụng size thực tế
                    except OSError:
                         print("   Warning: Could not get actual size of downloaded file.")
                         # Tiếp tục với size ban đầu hoặc 0

                    print(f"   Uploading [{file_name}] (Actual size: {filesize / (1024*1024):.2f} MB) from Colab to destination folder {dest_folder_id[:10]}...")
                    start_time_ul = time.time()
                    uploaded_file_info = None

                    # Định nghĩa API call để truyền vào hàm retry (cho việc upload)
                    def upload_api_call():
                        # Tạo đối tượng media upload từ file đã tải về
                        # resumable=True là mặc định và quan trọng cho file lớn/mạng không ổn định
                        media = MediaFileUpload(local_temp_path, mimetype=file_mime_type, chunksize=32*1024*1024, resumable=True)
                        # Tạo metadata cho file mới
                        file_metadata = {'name': file_name, 'parents': [dest_folder_id]}
                        # Thực hiện upload, yêu cầu trả về 'id, name, size'
                        request = drive_service.files().create(
                            body=file_metadata,
                            media_body=media,
                            supportsAllDrives=True,
                            fields='id, name, size'
                        )
                        # Theo dõi tiến trình upload (tùy chọn, có thể làm chậm 1 chút)
                        # response = None
                        # progress = tqdm(total=filesize, unit='B', unit_scale=True, desc=f"   UL {file_name[:20]}...", leave=False, disable=(filesize <= 0))
                        # while response is None:
                        #     status, response = request.next_chunk()
                        #     if status and filesize > 0:
                        #         progress.update(int(status.resumable_progress - progress.n))
                        # progress.close()
                        # return response # Trả về khi hoàn tất
                        # Hoặc đơn giản hơn:
                        return request.execute() # Thực thi và chờ hoàn tất

                    # Thực thi upload với retry
                    uploaded_file_info = self._execute_with_retry(upload_api_call)
                    end_time_ul = time.time()

                    if uploaded_file_info:
                        # Lấy size chính xác hơn từ file đã upload
                        uploaded_size = int(uploaded_file_info.get('size', 0))
                        effective_size = uploaded_size if uploaded_size > 0 else filesize # Ưu tiên size đã upload
                        effective_size_mb = effective_size / (1024 * 1024)

                        if effective_size > 0:
                             # Chỉ cộng size nếu upload thành công
                            self._total_size += effective_size_mb
                            duration_ul = end_time_ul - start_time_ul
                            speed_ul_mb = effective_size_mb / duration_ul if duration_ul > 0.001 else 0
                            print(f"   Upload complete for [{file_name}]. Size [{effective_size_mb:0.2f}] MB. Speed [{speed_ul_mb:.2f}] MB/s")
                        else:
                            print(f"   Upload complete for [{file_name}] (size 0 or unknown).")

                        # Kiểm tra giới hạn dung lượng sau khi upload thành công
                        if self._limit_size > 0 and self._total_size >= (self._limit_size * 1024):
                            # Xóa file tạm trước khi thoát
                            if os.path.exists(local_temp_path):
                                try: os.remove(local_temp_path)
                                except OSError: pass
                            self.on_total_size_exceeded(f"Total size {self._total_size:.2f} MB exceeds limit {self._limit_size} GB. Ending the program.")
                    else:
                        # Lỗi upload đã được log trong _execute_with_retry
                        print(f"   --- Upload failed for [{file_name}]. See previous error logs.")

                    # --- BƯỚC 3: DỌN DẸP FILE TẠM (LUÔN CHẠY SAU KHI THỬ UPLOAD) ---
                    if os.path.exists(local_temp_path):
                        try:
                            os.remove(local_temp_path)
                            # print(f"   Temporary file {os.path.basename(local_temp_path)} removed.")
                        except Exception as e:
                            print(f"   Warning: Could not remove temporary file {local_temp_path}: {e}")

            # === TRƯỜNG HỢP KHÔNG THỂ COPY VÀ CŨNG KHÔNG THỂ DOWNLOAD ===
            else:
                 print(f"--- Skipping file [{file_name}] (ID: {file_id}): Both Copying AND Downloading are disabled by owner or policy.")
                 return # Bỏ qua file này hoàn toàn

        # --- Xử lý trường hợp là THƯ MỤC (giữ nguyên logic gốc) ---
        else:
            print(f"\nProcessing folder: [{file_name}] (ID: {file_id})")
            existing_sub_folder_id = self.check_if_exists(drive_service, dest_folder_id, file_name, FOLDER_MIME_TYPE)
            sub_folder_id = existing_sub_folder_id

            if not existing_sub_folder_id:
                print(f"Creating subfolder [{file_name}] in destination.")
                # create_folder đã sử dụng _execute_with_retry nên đã có xử lý lỗi
                sub_folder_id = self.create_folder(drive_service, dest_folder_id, file_name)

            if sub_folder_id:
                print(f"Entering folder [{file_name}] (Destination ID: {sub_folder_id})...")
                # Lấy danh sách con từ thư mục nguồn (lấy tất cả con, không phân trang ở đây)
                source_files_in_folder = self.get_childs_from_folder(drive_service, file_id, 0, 0)
                if source_files_in_folder:
                    print(f"Found {len(source_files_in_folder)} items inside [{file_name}]. Starting processing...")
                    # Gọi đệ quy để sao chép/download/upload nội dung bên trong
                    self.copy_multiple_files(drive_service, sub_folder_id, source_files_in_folder)
                    print(f"Finished processing contents of folder [{file_name}].\n")
                else:
                     print(f"Source folder [{file_name}] is empty or no items fetched. Nothing to process inside.")
            else:
                # Lỗi đã được log trong create_folder hoặc check_if_exists (nếu có)
                print(f"Failed to create or find destination subfolder [{file_name}]. Skipping contents.")

    # ==============================================================
    # KẾT THÚC HÀM COPY_FILE ĐÃ SỬA
    # ==============================================================

    # ... (Các hàm create_folder, check_if_exists, copy_multiple_files, extract_folder_id_from_url, on_total_size_exceeded giữ nguyên như code gốc bạn cung cấp) ...

    def create_folder(self, drive_service: Resource, dest_folder_id: str, sub_folder_name: str) -> t.Optional[str]:
        """Tạo thư mục, kiểm tra tồn tại trước, có retry."""
        exist_folder_id = self.check_if_exists(drive_service, dest_folder_id, sub_folder_name, FOLDER_MIME_TYPE)
        if exist_folder_id:
             # print(f"Folder '{sub_folder_name}' already exists with ID: {exist_folder_id}. Using existing.")
             return exist_folder_id
        else:
            # print(f"Attempting to create folder: {sub_folder_name}")
            sub_folder_inf = {'name': sub_folder_name, 'mimeType': FOLDER_MIME_TYPE, 'parents': [dest_folder_id]}
            def create_api_call():
                return drive_service.files().create(
                    body=sub_folder_inf,
                    fields='id',
                    supportsAllDrives=True
                ).execute()
            folder_info = self._execute_with_retry(create_api_call) # Dùng retry đã sửa
            if folder_info and folder_info.get('id'):
                folder_id = folder_info.get('id')
                print(f"Folder '{sub_folder_name}' created successfully with ID: {folder_id}")
                return folder_id
            else:
                 # Lỗi đã được log trong _execute_with_retry
                 print(f"--- Failed to create folder [{sub_folder_name}]...")
                 return None

    def check_if_exists(self, drive_service: Resource, dest_folder_id: str, name: str, mime_type: t.Optional[str] = None) -> t.Optional[str]:
        """Kiểm tra sự tồn tại của tệp/thư mục với tên chính xác và tùy chọn mimeType."""
        try:
            # Escape các ký tự đặc biệt ' và \ trong tên file để dùng trong query
            processed_name = name.replace("\\", "\\\\").replace("'", "\\'")
            query = f"'{dest_folder_id}' in parents and name = '{processed_name}' and trashed=false"
            if mime_type:
                query += f" and mimeType = '{mime_type}'"

            # fields chỉ cần id là đủ để xác nhận tồn tại
            results = drive_service.files().list(
                q=query,
                fields='files(id)', # Chỉ cần ID là đủ
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                pageSize=1 # Chỉ cần 1 kết quả là đủ xác nhận
            ).execute()
            files = results.get('files', [])
            if files:
                return files[0]['id'] # Trả về ID nếu tìm thấy
        except HttpError as error:
             # Chỉ in lỗi nếu không phải 404 (Not Found là bình thường khi check)
             # Và cũng không phải lỗi 403 (Permission denied khi list folder đích cũng có thể xảy ra)
            if error.resp.status not in [404, 403]:
                 print(f"An HTTP error occurred while checking existence for '{name}': {error}")
            # Nếu lỗi 403 khi list folder đích, không thể check tồn tại -> Giả sử chưa tồn tại
            elif error.resp.status == 403:
                 print(f"Warning: Permission denied checking existence in folder {dest_folder_id}. Assuming '{name}' does not exist.")
                 return None
        except Exception as e:
            print(f"An unexpected error occurred while checking existence for '{name}': {e}")
        return None # Trả về None nếu không tìm thấy hoặc có lỗi khác

    def copy_multiple_files(self, drive_service: Resource, dest_folder_id: str, source_files: t.List[t.Dict[str, t.Any]]):
        """Sao chép nhiều tệp/thư mục với thanh tiến trình tqdm."""
        total_files = len(source_files)
        if total_files == 0:
            # print("No items to process in this batch.") # Giảm bớt log không cần thiết
            return
        # Lấy tên thư mục đích để hiển thị (nếu có thể)
        dest_folder_name = f"ID {dest_folder_id[:10]}..."
        # Cố gắng lấy tên thư mục đích, bỏ qua nếu lỗi
        try:
            dest_info = drive_service.files().get(fileId=dest_folder_id, fields='name', supportsAllDrives=True).execute()
            dest_folder_name = dest_info.get('name', dest_folder_name)
        except:
            pass

        # Sử dụng leave=False để progress bar biến mất sau khi xong batch này (nếu là subfolder)
        for source_file in tqdm(source_files, desc=f"Processing in '{dest_folder_name}'", unit="item", leave=False):
            self.copy_file(drive_service, dest_folder_id, source_file) # Gọi copy_file đã sửa
            # Giảm hoặc bỏ sleep nếu không gặp rate limit quá nhiều
            # time.sleep(0.02) # Thời gian chờ nhỏ giữa các file

    def extract_folder_id_from_url(self, url: str) -> t.Optional[str]:
        """Trích xuất ID thư mục/tệp từ URL Google Drive."""
        # Thêm pattern cho link dạng /u/0/..., /u/1/...
        patterns = [
            r'/folders/([-\w]{25,})', r'/drive/folders/([-\w]{25,})',
            r'/drive/u/\d+/folders/([-\w]{25,})', # Hỗ trợ /u/0/, /u/1/,...
            r'id=([-\w]{25,})',
            r'/file/d/([-\w]{25,})',
            r'/drive/u/\d+/files/d/([-\w]{25,})' # Hỗ trợ /u/0/, /u/1/,... cho file
             ]
        for pattern in patterns:
            match = re.search(pattern, url)
            if match: return match.group(1) # Trả về group 1 (ID)

        # Nếu không khớp các pattern trên, thử lấy phần cuối của path nếu nó giống ID
        if '/' in url:
            # Tách phần query string nếu có
            path_part = url.split('?')[0]
            potential_id = path_part.split('/')[-1]
            # Kiểm tra xem phần cuối có giống ID không (dài, không có dấu chấm)
            # Thêm điều kiện kiểm tra ký tự hợp lệ (chữ, số, -, _)
            if len(potential_id) >= 25 and re.match(r'^[-_\w]+$', potential_id) and potential_id != 'edit':
                 print(f"Attempting to use potential ID from URL path: {potential_id}")
                 return potential_id

        print(f"Warning: Could not automatically extract a valid ID from URL: {url}")
        print("Please provide a direct link to the folder/file or just the ID.")
        return None

    def on_total_size_exceeded(self, message: str):
        """Hàm xử lý khi vượt quá giới hạn dung lượng."""
        print(f"\n!!! LIMIT EXCEEDED !!!")
        print(message)
        print(f"Total size reached: {self._total_size / 1024:.3f} GB ({self._total_size:.2f} MB)")
        print("Exiting program.")
        sys.exit(1) # Thoát chương trình

    def copy_drive_to_drive(self, destDriveLink: str, sourceDriveLink: str, from_page: int, to_page: int):
        """Hàm chính điều phối quá trình sao chép."""
        print("Starting Drive to Drive copy process...")
        service = self.get_user_credential()
        if not service:
            print("Failed to get Google Drive credentials. Aborting.")
            return

        start_time_total = time.time()
        dest_folder_id_parent = self.extract_folder_id_from_url(destDriveLink)
        source_folder_id = self.extract_folder_id_from_url(sourceDriveLink)

        if not dest_folder_id_parent or not source_folder_id:
            print(f"Error: Could not extract valid folder/file IDs from URLs.")
            if not dest_folder_id_parent: print(f"  Check Destination Link/ID: {destDriveLink}")
            if not source_folder_id: print(f"  Check Source Link/ID: {sourceDriveLink}")
            return

        print(f"Source Item ID: {source_folder_id}")
        print(f"Destination Parent Folder ID: {dest_folder_id_parent}")
        if self._limit_size > 0: print(f"Effective Size Limit: {self._limit_size} GB")
        else: print("Effective Size Limit: Unlimited")
        if self.excluded_strings: print(f"Excluding items with names containing: {self.excluded_strings}")

        try:
            print(f"Fetching source item info...")
            # Kiểm tra xem source là file hay folder
            def get_source_info_call():
                 return service.files().get(fileId=source_folder_id, supportsAllDrives=True, fields='id, name, mimeType, capabilities, size').execute()
            source_info = self._execute_with_retry(get_source_info_call)

            if not source_info:
                 print(f"Error: Could not fetch source item information (ID: {source_folder_id}). Check ID and permissions. Aborting.")
                 return

            source_name = source_info.get('name', 'Unknown_Source_Item')
            source_mime_type = source_info.get('mimeType')
            print(f"Source item: '{source_name}' (Type: {source_mime_type})")

            # --- XỬ LÝ NẾU NGUỒN LÀ MỘT FILE DUY NHẤT ---
            if source_mime_type != FOLDER_MIME_TYPE:
                print(f"Source is a single file. Copying '{source_name}' directly into destination parent folder {dest_folder_id_parent[:10]}...")
                # Gọi copy_file để xử lý file duy nhất này, đích là thư mục cha đã cho
                self.copy_file(service, dest_folder_id_parent, source_info)

            # --- XỬ LÝ NẾU NGUỒN LÀ MỘT THƯ MỤC ---
            else:
                print(f"Source is a folder. Ensuring destination folder '{source_name}' exists inside parent {dest_folder_id_parent[:10]}...")
                # Tạo thư mục đích chính (tên giống thư mục nguồn) bên trong thư mục cha đã cho
                main_dest_folder_id = self.create_folder(service, dest_folder_id_parent, source_name)

                if not main_dest_folder_id:
                     print(f"Error: Failed to create/find main destination folder '{source_name}' inside {dest_folder_id_parent}. Aborting.")
                     return

                print(f"Destination folder ready: '{source_name}' (ID: {main_dest_folder_id})")

                print(f"Fetching item list from source folder '{source_name}'...")
                # Lấy danh sách các mục con từ thư mục nguồn, áp dụng phân trang
                source_files = self.get_childs_from_folder(service, source_folder_id, from_page, to_page)

                if source_files:
                    print(f"Starting to process {len(source_files)} items found in '{source_name}' (within page range)...")
                    # Sao chép/download/upload các mục con vào thư mục đích chính vừa tạo/tìm thấy
                    self.copy_multiple_files(service, main_dest_folder_id, source_files)
                else:
                    print("No files or folders found in the source directory (within specified page range) to process.")

        except HttpError as error:
            print(f"\nA critical HTTP error occurred during the process: {error}")
            status_code = error.resp.status
            if status_code == 404:
                print("Error 404: Please double-check if the Source or Destination IDs are correct and exist.")
            elif status_code == 403:
                 print("Error 403: Permission denied. Please check if you have sufficient permissions for both source and destination.")
            # Có thể thêm các xử lý lỗi cụ thể khác
        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()

        end_time_total = time.time()
        total_duration_seconds = end_time_total - start_time_total
        total_duration_minutes = total_duration_seconds / 60
        size_gb = self._total_size / 1024
        speed_mb_avg = self._total_size / total_duration_seconds if total_duration_seconds > 0.01 else 0

        print("\n================== SUMMARY ==================")
        print(f"Copy process finished.")
        print(f"Total Size Processed (Copied/Uploaded): {size_gb:.3f} GB ({self._total_size:.2f} MB)")
        print(f"Total Time: {total_duration_seconds:.2f} seconds ({total_duration_minutes:.2f} minutes)")
        if self._total_size > 0:
            print(f"Average Speed: {speed_mb_avg:.2f} MB/s (based on successful operations)")
        # Xác định đích cuối cùng dựa trên nguồn là file hay folder
        final_dest_description = f"Item '{source_name}' in folder ID {dest_folder_id_parent}" if source_mime_type != FOLDER_MIME_TYPE else f"Folder '{source_name}' (ID: {main_dest_folder_id if 'main_dest_folder_id' in locals() and main_dest_folder_id else 'N/A'}) inside parent ID {dest_folder_id_parent}"
        print(f"Destination: {final_dest_description}")
        print("===========================================")


# ==============================================================
# PHẦN MAIN ĐỂ CHẠY TRONG COLAB (GIỮ NGUYÊN)
# ==============================================================

# --- Phần lấy input từ widgets và khởi tạo ---
destDriveLink = "" #@param {type:"string"}
sourceDriveLink = "" #@param {type:"string"}
#@markdown ---
#@markdown **Optional Settings:**
#@markdown *Specify page range if source folder has many items (0 for all):*
fromPage = 0 #@param {type:"integer"}
toPage = 0 #@param {type:"integer"}
#@markdown *Limit total size copied/uploaded (GB, 0 for unlimited):*
limit_gb_input = 0.0 #@param {type:"number"}
#@markdown *Exclude items whose names contain these strings (comma-separated):*
exclude_input = "" #@param {type:"string"}


# --- TẠO INSTANCE VÀ CHẠY ---
downloader = DownloadFromDrive()
downloader._limit_size = limit_gb_input if limit_gb_input >= 0 else 0.0 # Đảm bảo không âm
downloader.excluded_strings = [ext.strip().lower() for ext in exclude_input.split(",") if ext.strip()] # Chuyển thành chữ thường để so sánh không phân biệt hoa thường

# Kiểm tra các biến đầu vào
if not destDriveLink or not sourceDriveLink:
    print("\n" + "="*40)
    print("!!! ERROR: Please provide valid Source and Destination Google Drive links/IDs.")
    print("Link example: https://drive.google.com/drive/folders/YOUR_FOLDER_ID")
    print("Or just provide the ID: YOUR_FOLDER_ID")
    print("="*40 + "\n")
# elif not downloader.extract_folder_id_from_url(destDriveLink) or not downloader.extract_folder_id_from_url(sourceDriveLink):
#     # Thêm kiểm tra ID hợp lệ sau khi extract (đã tích hợp vào hàm chính)
#     print("\n" + "="*40)
#     print("!!! ERROR: Could not extract valid IDs from the provided links.")
#     print("Please ensure the links are correct or provide IDs directly.")
#     print("="*40 + "\n")
else:
    # Chuyển fromPage, toPage thành số nguyên an toàn
    try:
        from_page_int = int(fromPage) if fromPage >= 0 else 0
    except (ValueError, TypeError):
        from_page_int = 0
    try:
        to_page_int = int(toPage) if toPage >= 0 else 0
    except (ValueError, TypeError):
        to_page_int = 0

    if to_page_int > 0 and from_page_int > to_page_int:
        print("\nWarning: 'From Page' is greater than 'To Page'. Setting 'To Page' to 0 (process all pages from 'From Page').")
        to_page_int = 0
    if from_page_int < 0 : from_page_int = 0 # Đảm bảo không âm
    if to_page_int < 0 : to_page_int = 0   # Đảm bảo không âm


    # Gọi hàm chính với các giá trị đã xử lý
    downloader.copy_drive_to_drive(destDriveLink, sourceDriveLink, from_page_int, to_page_int)
